# Jupyter Notebooks - part 2
* ### Data analysis and visualization

In this lesson you will learn 
- to do some data analysis using pandas
 - The important concept of *tidy* data
- to produce nice plots with seaborn
- how other types of media can be embedded in a notebook

## Exploratory data analysis in Jupyter

We will use four important Python packages
1. `numpy` is the fundamental package for scientific computing with Python
 - for the purposes of this notebook `numpy` is like the engine of a car, it's under the hood running the show
2. `pandas` is a more recently developed package for data manipulation and analysis 
 - powerful high-level tool for data exploration
 - two fundamental data structures which can be applied to many types of data: `Series` and `DataFrames`  

3. `matplotlib` is the standard Python package for plotting, "grandfather of all Python visualization packages"
4. `seaborn` is a higher-level visualization package based on `matplotlib`
 - while `matplotlib` is extremely powerful, it can also be complex. 
 - sometimes much effort needed to get good-looking graphs
 - default visualization much more appealing compared to `matplotlib`

We will download and process a dataset on Nobel prizes. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pandas defines a `read_csv` function that can read any CSV file. By giving the URL to the file, pandas will automatically download and parse the file, and return a `DataFrame` object. We need to specify a few options to make sure the dates are parsed correctly.

In [ ]:
# dataset from http://oppnadata.se/en/dataset/nobel-prizes/resource/f3da8ba9-a17f-4911-9003-4bcef93619cc
nobel = pd.read_csv("data/nobels.csv")

The `nobel` variable now contains a `DataFrame` object, a Pandas data structure that contains 2D tabular data. The `head(n)` method displays the first `n` rows of this table.

In [ ]:
nobel.head()

Each column (and row) of the `DataFrame` is a `Series`. Series can be accessed by their names as follows.

In [ ]:
nobel["year"]

In [ ]:
type(nobel["year"])

A Series object can produce statistical information about the datums in it.

In [ ]:
nobel["share"].describe()

It's also somewhat smart about the contents of the data it sees so it can summarize non-numerical data as well.

In [ ]:
nobel["bornCountryCode"].describe()

If you call a method on the dataframe like count, it will call the same method on each of the series.

In [ ]:
nobel.count()

The dataset is clearly not quite complete, especially in the death statistics. Possibly because the laureates are still alive?

We can also use the function `describe()` to request statistics for the entire dataframe but then it will only give statistics for the numerical variables.

In [ ]:
nobel.describe()

To calculate some more elaborate statistics, we first add a column (one Nobel prize per laureate). This will add the column "number" to the dataframe with the value 1 for each row.

In [ ]:
nobel["number"] = 1

### Age statistics

Let's first look at statistics based on the age of prize recipients.  
We need to convert the "born" column to datetime format. Datetimes are hardly ever recognized correctly.

In [ ]:
type(nobel["born"][0])

In [ ]:
nobel["born"] = pd.to_datetime(nobel["born"], errors ='coerce')
# coercion is necessary because the data is a bit messy

In [ ]:
type(nobel["born"][0])

In [ ]:
nobel["born"].dt.year

We can now add a column to the DataFrame with age when prize was received.

In [ ]:
nobel["age"] = nobel["year"] - nobel["born"].dt.year
nobel[["surname","age"]].head(10)
#print(nobel["age"].to_string())

We can now plot a histogram of the age at which laureates receive their prize, using the inbuilt matplotlib support of pandas 

In [ ]:
nobel.plot?

In [ ]:
nobel["age"].plot.hist(bins=[20,30,40,50,60,70,80,90,100],alpha=0.6);

To extract the numbers, use the value_counts method

In [ ]:
nobel["age"].value_counts(bins=[20,30,40,50,60,70,80,90,100])

An alternative plot that is is better for comparing distributions is the box plot.

The "by" keyword tells by which value the the observations should be **grouped by**, which is the next topic.

In [ ]:
nobel.boxplot(column="age", by="category")

### Country statistics - groupby

We use the powerful `groupby` method to split data into groups, select the column "number", and sum up to get the total sum of Nobel prizes by country 

In [ ]:
nobels_by_country = nobel.groupby('bornCountry',sort=True)["number"].sum()


In [ ]:
nobels_by_country.describe()

The pandas Series only shows a limited number of rows. Let's print them all

In [ ]:
print(nobels_by_country.to_string())

How many prizes have people born in Sweden received?

In [ ]:
nobels_by_country["Sweden"]

Who were they?

In [ ]:
nobel.loc[nobel['bornCountry'] == "Sweden"]

We move on. Let's extract four countries and generate some plots

In [ ]:
countries = np.array(["France", "USA", "United Kingdom", "Sweden"])

In [ ]:
nobel2 = nobel.loc[nobel['bornCountry'].isin(countries)]

We now group by both `bornCountry` and `category`

In [ ]:
nobels_by_country2 = nobel2.groupby(['bornCountry',"category"],sort=True).sum()
nobels_by_country2["number"].head(50)

We can reshape the `DataFrame` a bit using the pivot_table method to create a spreadsheet-like representation

In [ ]:
table = nobel2.pivot_table(values="number",index="bornCountry", columns="category",aggfunc=np.sum)
table

This representation can be used to make a heatmap visualization

### <font color="red"> *Exercise 2.1* </font>

Gripped by literally morbid curiosity you want to know where Nobel laureates died and presumably are buried.

Group the laureates by country of death, use `sort_values` function of pandas.DataFrame to sort them in ascending order and select the 5 countries with most deaths.


In [ ]:
nobels_by_country_of_death = nobel.groupby('diedCountry',sort=True)["number"].sum()
nobels_by_country_of_death.sort_values(ascending=False).head(5)

### Single country statistics - filter

Often the characteristics of a single subset are interesting.

To do this we usually use some Python syntax that may look a bit strange at first. 

See what the following command produces. The `.head(5)` is just there to limit the results to the first 5.

In [ ]:
nobel.head(5)["bornCountryCode"] == "NL"

It uses some Python syntactic sugar to create a Series of boolean variables, True for the rows for which `["bornCountryCode"] == "NL"` is true and False for the rest.

If we pass this Series back to the DataFrame using the []-brackets we get all those rows from the DataFrame that were True in the Series

In [ ]:
dutch_nobelists = nobel[nobel["bornCountryCode"] == "NL"]

It is also possible to combine the clauses for more complex filters/queries.

In [ ]:
born_and_died_in_sweden = nobel[(nobel["bornCountryCode"] == "SE") 
                                & (nobel["diedCountryCode"] == "SE")]


### <font color="red"> *Exercise 2.2* </font>

Find out which Swedish Nobel prize winner did not die in Sweden.

Hints: 
- the change is a very minor one, don't think too complicated
- a NaN means that either the person is still alive or their place of death isn't known)
 - the `dropna` function can help you filter those but that is not strictly necessary


In [ ]:
#write your answer here


### Visualizing data with seaborn

In [ ]:
sns.heatmap(table,linewidths=.5, annot=True);

Violin plots can reveal trends (but are not very quantitative)

In [ ]:
sns.violinplot(y="year", x="bornCountry",inner="stick", data=nobel2);

We can also use multiple conditions. Let's extract only physics prizes

In [ ]:
nobel3 = nobel.loc[nobel['bornCountry'].isin(countries) & nobel['category'].isin(['physics'])]
sns.violinplot(y="year", x="bornCountry",inner="stick", data=nobel3);

Swarmplots display categorical scatterplots with non-overlapping points

In [ ]:
sns.swarmplot(y="year", x="bornCountry", data=nobel2, alpha=.5);

In [ ]:
sns.factorplot(x="bornCountry", y="year", col="category", data=nobel2, kind="swarm");

In [ ]:
sns.factorplot(x="bornCountry", col="category", data=nobel2, kind="count");

Seaborn uses a variant of the The Grammar of Graphics, on which most famously simple graphics libraries are based. This is beyond the scope of this notebook though.

### Other visualization packages
* [Plotly](https://plot.ly/) - commercial online service for creating and sharing visualizations in notebooks
* [Bokeh](http://bokeh.pydata.org/en/latest/) - web-based, general-purpose and fast visualization toolkit
* [mpld3](http://mpld3.github.io/examples/index.html) - must be seen...

# Tidy Data

You may have observed that the data analysis seemed ridiculously easy in the example. This is in fact quite true, because the data was already **in the right format**.

It is said that 80% of a data analysts time is spent on the gritty details of understanding data and getting it  to the right format and the other 20% on the actual analysis.

Both Pandas in Python and the Tidyverse packages in R ascribe to the concept of **tidy** data as presented by Hadley Wickham. The original article on it can be found [here](https://www.jstatsoft.org/article/view/v059i10) and is worth a read.

To summarize data is tidy when

1. each variable forms a column
2. each observation forms a row
3. each type of observational unit forms a table

What an observation is and what a variable is depend on the semantics of the analytics question you are facing.


## Melting

Let's assume we have some run time statistics from a 1500 m running event.

| Runner   | 400m | 800m | 1200m | 1500m |
|----------|------|------|-------|-------|
| Runner 1 | 64   | 128  | 192   | 240   |
| Runner 2 | 80   | 160  | 240   | 300   |
| Runner 3 | 96   | 192  | 288   | 360   |

This is a classical table generated for displaying information. The issue here is that the column names 400m, 800m, 1200m and 1500m are, in fact variables.

To tidy the data we'd like it to be in the following format.

| Runner   | distance | time(s) |
|----------|----------|---------|
| Runner 1 | 400m     | 64      |
| Runner 1 | 800m     | 128     |
| ....     | ...      | ...     |

That way we can perform  aggregate operations on it, particularly we can **filter**  and **group** the data set. The data is also in a format where it is possible to model relationships between variables.

In [ ]:
df = pd.DataFrame([
        {'Runner': 'Runner 1', 400: 64, 800: 128, 1200: 192, 1500: 240},
        {'Runner': 'Runner 2', 400: 80, 800: 160, 1200: 240, 1500: 300},
        {'Runner': 'Runner 3', 400: 96, 800: 192, 1200: 288, 1500: 360},
         ])
df

In [ ]:
df = pd.melt(df, id_vars="Runner", 
             value_vars=[400, 800, 1200, 1500], 
             var_name="distance", 
             value_name="time"
            )
df

Now, to e.g. compute the time spent on each interval, we c

In [ ]:
df.sort_values("distance").groupby("Runner").time.diff()

### <font color="red"> *Exercise 2.2* </font>
Go ahead perform a similar melt on the data below to make it **tidy**.

In [ ]:
df = pd.DataFrame([
        {"Factory": "Factory 1", 1998: 3, 1999: 10, 2000: 5},
        {"Factory": "Factory 2", 1998: 50, 1998: 35, 1999: 100, 2000: 0},
        {"Factory": "Factory 3", 1999: 20, 1998: 20, 1999: 20, 2000: 25},
         ])


## Merging

The database world has the concept of joins and tidy data sometimes needs to be joined as well to create a larger DataFrame with redundant data.

In Pandas the function to do this is called `merge`.

In [ ]:
orders = pd.DataFrame([
        {"Person": "Dick", "Pizza": "Pepperoni"},
        {"Person": "Tom", "Pizza": "Hawaii"},
        {"Person": "Harry", "Pizza": "Capricciosa"}])

toppings = pd.DataFrame([
        {"Pizza": "Pepperoni", "Ingredient 1": "Pepperoni", "Ingredient 2": "Cheese"},
        {"Pizza": "Margherita", "Ingredient 1": "Cheese", "Ingredient 2": "Tomato"},
        {"Pizza": "Hawaii", "Ingredient 1": "Ham", "Ingredient 2": "Pineapple"},
        {"Pizza": "Capricciosa", "Ingredient 1": "Mushrooms", "Ingredient 2": "Ham"},
])
    

In [ ]:
person_toppings = orders.merge(toppings, on="Pizza")
person_toppings

The default join type is an **inner** join. We could also do a **left**, **outer** or **right** join.

If the concepts are not familiar don't worry, you'll run across them sooner or later.

In [ ]:
orders.merge(toppings, on="Pizza", how="outer")


### <font color="red"> *Exercise 2.3* </font>
The abovementioned pizza data is not yet **tidy** because the column labels "Ingredient 1" and "Ingredient 2" are in fact priority values.

Make a tidy version of person_toppings. Who have ordered pizzas that have ham in it?

## Other concepts

This tutorial does not aim to be complete. We only have enough time to show you the way.

Other important concepts for tidying data are
* **splitting** data when a single column holds multiple variables
 * e.g. "male10-18" contains two variables, gender and age group
* parsing dates, timestamps and other nontrivial datums


## Other types of media

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('j9YpkSX7NNM')

In [ ]:
from IPython.display import Audio
Audio("data/GW150914_L1_whitenbp.wav")

In [ ]:
from IPython.display import IFrame
IFrame("http://jupyter.org",width='100%',height=350)